In [0]:
!pip3 install 'torch==0.4.0'
!pip3 install 'torchvision==0.2.1'
!pip3 install 'numpy'
!pip3 install 'Pillow'

    100% |████████████████████████████████| 484.0MB 37kB/s 
fastai 1.0.51 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2
    100% |████████████████████████████████| 61kB 2.3MB/s 
fastai 1.0.51 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torchvision 0.2.2.post3
    Uninstalling torchvision-0.2.2.post3:
      Successfully uninstalled torchvision-0.2.2.post3


In [0]:
from PIL import Image
import random
import math

import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

from google.colab import drive
drive.mount('/drive')

np.random.seed(123)
random.seed(123)
torch.manual_seed(123)


class RandomErasing(object):
    '''
    Class that performs Random Erasing in Random Erasing Data Augmentation by Zhong et al. 
    -------------------------------------------------------------------------------------
    probability: The probability that the operation will be performed.
    sl: min erasing area
    sh: max erasing area
    r1: min aspect ratio
    mean: erasing value
    source: https://github.com/zhunzhong07/Random-Erasing/blob/master/transforms.py
    -------------------------------------------------------------------------------------
    '''
    def __init__(self, probability = 0.5, sl = 0.02, sh = 0.4, r1 = 0.3, mean=[0.4914, 0.4822, 0.4465]):
        self.probability = probability
        self.mean = mean
        self.sl = sl
        self.sh = sh
        self.r1 = r1

    def __call__(self, img):

        if random.uniform(0, 1) > self.probability:
            return img

        for attempt in range(100):
            area = img.size()[1] * img.size()[2]

            target_area = random.uniform(self.sl, self.sh) * area
            aspect_ratio = random.uniform(self.r1, 1/self.r1)

            h = int(round(math.sqrt(target_area * aspect_ratio)))
            w = int(round(math.sqrt(target_area / aspect_ratio)))

            if w < img.size()[2] and h < img.size()[1]:
                x1 = random.randint(0, img.size()[1] - h)
                y1 = random.randint(0, img.size()[2] - w)
                if img.size()[0] == 3:
                    img[0, x1:x1+h, y1:y1+w] = self.mean[0]
                    img[1, x1:x1+h, y1:y1+w] = self.mean[1]
                    img[2, x1:x1+h, y1:y1+w] = self.mean[2]
                else:
                    img[0, x1:x1+h, y1:y1+w] = self.mean[0]
                return img

        return img

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive


In [0]:
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                            torchvision.transforms.ToTensor()])

train_set = torchvision.datasets.CIFAR10(root='./data/', train=True,
                                        download=True, transform=transform)

### Applying augumentation

In [0]:
train_mean = np.mean(train_set.train_data, axis=(0,1,2)) / 255
train_std = np.std(train_set.train_data, axis=(0,1,2)) / 255


augumentation_transforms = [torchvision.transforms.RandomResizedCrop(224, scale=(0.7, 1)),
                            torchvision.transforms.RandomAffine(translate=(0.1, 0.1), degrees=(-15,15)),
                            torchvision.transforms.ColorJitter(brightness=0.3,
                                                               contrast=0.5,
                                                               saturation=0.5,
                                                               hue=0.1),
                            torchvision.transforms.RandomHorizontalFlip(),
                            torchvision.transforms.ToTensor(),
                            RandomErasing(probability=0.5, sl=0.02, sh=0.2, r1=0.3, mean=train_mean)]

transform_train = torchvision.transforms.Compose(augumentation_transforms + 
                                                 [torchvision.transforms.Normalize(train_mean, train_std)])
transform_test = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                                torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Normalize(train_mean, train_std)])


train_set = torchvision.datasets.CIFAR10(root='./data/', train=True,
                                        download=False, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64,
                                         shuffle=False, num_workers=2)

### Downloading and inititalizating ResNet

In [0]:
resnet = torchvision.models.resnet101(pretrained=True)
# resnet = torchvision.models.resnet101(pretrained=False)
resnet_num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(resnet_num_ftrs, 10)

# resnet.load_state_dict(torch.load('/drive/My Drive/resnet101randomcrop.pth'))
resnet.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

### Training

In [0]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

losses = []
accuraccies = []

n_epochs = 20

for epoch in range(0, n_epochs):
    scheduler.step()
    loss_train = 0
    corrects = 0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        predictions = resnet(inputs)
        loss = loss_func(predictions, labels)
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
        _, pred_labels = torch.max(predictions.data, 1)
        corrects += (pred_labels == labels.cuda()).sum().item()
    loss /= len(train_loader.dataset)
    accuracy = corrects / len(train_loader.dataset)
    print ("[{}/{}] loss: {}, accuracy: {}".format(epoch + 1, n_epochs, loss_train, accuracy))
    losses.append(loss)
    accuraccies.append(accuracy)

[1/20] loss: 1.365844946121797e-05, accuracy: 0.79864
[2/20] loss: 9.312758265878074e-06, accuracy: 0.91404
[3/20] loss: 1.6584204786340706e-05, accuracy: 0.9308
[4/20] loss: 2.0744155335705727e-06, accuracy: 0.9419
[5/20] loss: 2.8447664135455852e-06, accuracy: 0.94876
[6/20] loss: 7.129565346986055e-06, accuracy: 0.95506
[7/20] loss: 1.1697708032443188e-05, accuracy: 0.95786
[8/20] loss: 1.1743684808607213e-05, accuracy: 0.96706
[9/20] loss: 4.220212758809794e-06, accuracy: 0.9692
[10/20] loss: 4.858363809034927e-06, accuracy: 0.97148
[11/20] loss: 2.006289469136391e-05, accuracy: 0.97282
[12/20] loss: 1.0416703844384756e-05, accuracy: 0.97282
[13/20] loss: 5.25604582435335e-06, accuracy: 0.97308
[14/20] loss: 2.414020173091558e-06, accuracy: 0.97276
[15/20] loss: 6.690808277198812e-06, accuracy: 0.97436
[16/20] loss: 8.313754733535461e-06, accuracy: 0.97432
[17/20] loss: 1.822447757149348e-07, accuracy: 0.97506
[18/20] loss: 2.910274133682833e-06, accuracy: 0.97426
[19/20] loss: 3.4

# Accuracy on test set

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = resnet(images.cuda())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum().item()

print('Accuracy on test: {}'.format(correct / total))

Accuracy on test: 0.9721


Saving model to the drive

In [0]:
 torch.save(resnet.state_dict(), '/drive/My Drive/resnet101final.pth') 